In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Importing dependencies
import pandas as pd
from sodapy import Socrata
from config import app_token
from config import username
from config import password

#dependencies for 2nd data source
import pandas as pd
import scipy.stats as st
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

ModuleNotFoundError: No module named 'sodapy'

In [ ]:
# Getting the data from API NY Open Data
client = Socrata("data.cityofnewyork.us",
                  app_token, username,
                  password)
# Database returned as JSON from API / converted to Python list of dictionaries by sodapy.
results = client.get("gaq9-z3hz", limit=2832)

# Convert to pandas DataFrame
Recycling_Div_Captures = pd.DataFrame.from_records(results)

In [ ]:
# creating path for the citywide subsort 
data_path = "Resources/Citywide_Subsort.csv"


# Read the data into df
cws_df = pd.read_csv(data_path)


#Preview data
cws_df.head()

In [ ]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://data.cityofnewyork.us/City-Government/DSNY-Waste-Characterization-Mainsort/k3ks-jzek'
browser.visit(url)

In [ ]:
df_list = []
for x in range(1,40):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    table = soup.find_all('div', class_='socrata-table frozen-columns')
    df_list.append(pd.read_html(str(table))[0])
    button = browser.find_by_xpath('//*[@id="app"]/div/div[2]/section[4]/div[2]/div/div/div[4]/div[3]/span[1]/button[2]').click()

In [ ]:
df_combined = pd.concat(df_list)

In [ ]:
df_combined.reset_index(drop=True, inplace=True)

In [ ]:
df_combined.head(20)